### Counts gloss instances of the file

In [3]:
import pandas as pd
from glob import glob

# Read all gloss files
words = []
for path in glob("../txt/lsl_glosses_*.txt"):
    with open(path, "r", encoding="utf-8") as f:
        words.extend(f.read().split())

df = pd.DataFrame(words, columns=["word"])

word_counts = df["word"].value_counts().reset_index()
word_counts.columns = ["word", "count"]

filtered = word_counts[word_counts["count"] <= 1]

print(filtered.to_string(index=False))

     word  count
     lēts      1
 darbnīca      1
     reti      1
     kurš      1
    jauns      1
  lielums      1
     kopt      1
  violets      1
    melns      1
 produkts      1
    glīts      1
      īss      1
diennakts      1
       20      1
 izmantot      1
